In [1]:
import os

In [2]:
os.getcwd()

'c:\\Users\\Ansh Lulla\\VS-Code\\MLOps\\16-End-to-end-Fine-Tuning\\research'

In [3]:
os.chdir("../")
os.getcwd()

'c:\\Users\\Ansh Lulla\\VS-Code\\MLOps\\16-End-to-end-Fine-Tuning'

In [17]:
from src.textSummarizer.logging import logging
from dataclasses import dataclass
from pathlib import Path
from src.textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.textSummarizer.utils.common import read_yaml, create_directories

In [25]:
print("Config path:", CONFIG_FILE_PATH)
print("Exists:", CONFIG_FILE_PATH.exists())

Config path: config\config.yaml
Exists: True


In [22]:
contents = read_yaml(CONFIG_FILE_PATH)
contents.artifacts_root

[2025-06-28 03:40:51,181: INFO: common: yaml file: config\config.yaml loaded successfully]


'artifacts'

In [19]:
@dataclass
class DataIngestionConfig:
  root_dir: Path
  source_URL: Path
  local_data_file: Path
  unzip_dir: Path

In [49]:
class ConfigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH, params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [50]:
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger

In [51]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} has been downloaded successfully")
        else:
            logger.info(f"File already exists")
    
    def unzip_file(self):
        unzip_dir = self.config.unzip_dir
        os.makedirs(unzip_dir, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, "r") as file:
            file.extractall(unzip_dir)

In [52]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(data_ingestion_config)
data_ingestion.download_file()
data_ingestion.unzip_file()

[2025-06-28 04:05:14,594: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-28 04:05:14,596: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-28 04:05:14,597: INFO: common: Created Directory at: artifacts]
[2025-06-28 04:05:14,599: INFO: common: Created Directory at: artifacts/data_ingestion]
[2025-06-28 04:05:19,280: INFO: 3276253235: artifacts/data_ingestion/data.zip has been downloaded successfully]
